In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [2]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_202005_consent_en.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/consent.md"

In [5]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../original/consent.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [6]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)(?!\.)\s*(.*)') 
point_pattern = re.compile(r'^(\d\.\d\.\d)\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX -- (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = point
            data.append([section, subsection, point, heading, text, section_reference])

        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0,1,2,3,4, 5])
df.reset_index(drop=True, inplace=True)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [7]:
df[0:20]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
0,0,,,True,PREFACE,0
1,,,,False,On 10 April 2018 the Article 29 Working Party ...,0
2,,,,False,The EDPB has noticed that there was a need for...,0
3,,,,False,1 The validity of consent provided by the ...,0
4,,,,False,2 The example 16 on scrolling and consent.,0
5,,,,False,The paragraphs concerning these two issues hav...,0
6,,,,False,- Section on Conditionality (paragraphs 38 - 4...,0
7,,,,False,- Section on Unambiguous indication of wishes ...,0
8,1,,,True,INTRODUCTION,1
9,,,,False,1. These Guidelines provide a thorough analysi...,1


In [8]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 10 augmented with footnotes
Row 11 augmented with footnotes
Row 12 augmented with footnotes
Row 13 augmented with footnotes
Row 14 augmented with footnotes
Row 15 augmented with footnotes
Row 18 augmented with footnotes
Row 26 augmented with footnotes
Row 27 augmented with footnotes
Row 28 augmented with footnotes
Row 32 augmented with footnotes
Row 36 augmented with footnotes
Row 37 augmented with footnotes
Row 38 augmented with footnotes
Row 42 augmented with footnotes
Row 47 augmented with footnotes
Row 48 augmented with footnotes
Row 53 augmented with footnotes
Row 56 augmented with footnotes
Row 75 augmented with footnotes
Row 80 augmented with footnotes
Row 81 augmented with footnotes
Row 91 augmented with footnotes
Row 92 augmented with footnotes
Row 93 augmented with footnotes
Row 94 augmented with footnotes
Row 95 augmented with footnotes
Row 96 augmented with footnotes
Row 100 augmented with footnotes
Row 101 augmented with footnotes
Row 103 augmented with footnotes
Row 1

In [1]:
file = "../inputs/documents/consent.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


NameError: name 'df' is not defined

## Check that the document class works as expected

In [2]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.consent
importlib.reload(gdpr_rag.documents.consent)
from gdpr_rag.documents.consent import Consent

path_to_manual_as_csv_file = "../inputs/documents/consent.parquet"
doc = Consent(path_to_manual_as_csv_file)


In [18]:
section = "1"
section = "8"
section = "3.3.2"
#section = "Annex"

print(doc.get_heading(section))
print(doc.get_text(section))


3 ELEMENTS OF VALID CONSENT 
3.3 Informed 
3.3.2 How to provide information
3 ELEMENTS OF VALID CONSENT 
3.3 Informed 

3.3.2 How to provide information 
66. The GDPR does not prescribe the form or shape in which information must be provided in order to fulfil the requirement of informed consent. This means valid information may be presented in various ways, such as written or oral statements, or audio or video messages. However, the GDPR puts several requirements for informed consent in place, predominantly in Article 7(2) and Recital 32. This leads to a higher standard for the clarity and accessibility of the information. 
67. When seeking consent, controllers should ensure that they use clear and plain language in all cases. This means a message should be easily understandable for the average person and not only for lawyers. Controllers cannot use long privacy policies that are difficult to understand or statements full of legal jargon. Consent must be clear and distinguishable from